In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
!pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 227.3 kB/s eta 0:00:00


In [3]:
from datasets import load_dataset

In [4]:
pd.set_option('display.max_colwidth', None)

NameError: name 'pd' is not defined

In [ ]:
ds = load_dataset("m-a-p/MusicPile-sft")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1139473 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'src', 'id'],
        num_rows: 1139473
    })
})

In [ ]:
# prompt: Convert ds to a pandas df

import pandas as pd

pdf = pd.DataFrame(list(pd.DataFrame(ds)["train"]))
pdf

,instruction,input,output,src,id
0,Human: Design melodic orchestrations by follow...,"'A', 'D', 'E7', 'A', 'E/G#', 'A', 'Bm', 'A7/C#...","Assistant: X:1\nL:1/8\nM:2/2\nK:A\n CS ||""A"" A...",https://huggingface.co/datasets/sander-wood/ir...,0
1,Human: Compose a brief account using the detai...,title:青春常駐\tartist:Hins Cheung\talbum:Morph\td...,"Assistant: ""Eternal Spring"" is a song by artis...",distilled,1
2,Human: Summarize the information found in the ...,title:Con Cronin's\tartist:None\talbum:None\td...,"Assistant: ""Con Cronin's"" is a lively polka pi...",distilled,2
3,Human: Craft a short synopsis using the data f...,title:Aasai Aasai\tartist:Maragathamani\talbum...,"Assistant: ""Aasai Aasai"" is a song from the so...",distilled,3
4,Human: Construct an overview by drawing on the...,title:Nothing To Do With Love\tartist:Kenny Wa...,"Assistant: ""Nothing To Do With Love"" is a trac...",distilled,4
...,...,...,...,...,...
1139468,Human: Develop a compact description based on ...,title:Wvsted\tartist:I.Y.F.F.E\talbum:Wvsted\t...,"Assistant: ""Wvsted"" is a song by the artist I....",distilled,1139468
1139469,"Human: Discuss the role of repetition, variati...",,"Assistant: In music, repetition, variation, an...",distilled,1139469
1139470,Human: Craft a concise summary using the detai...,title:The $48 Firework\tartist:Jim Breuer\talb...,"Assistant: ""The $48 Firework"" is a comedy trac...",distilled,1139470
1139471,Human: Look into the music with great attentio...,X:1\nL:1/8\nM:4/4\nK:A\n AB | cE E2 c2 cB | Ac...,Assistant: cE E2 c2 </s>,https://huggingface.co/datasets/sander-wood/ir...,1139471


In [ ]:
filtered_source_pdf = pdf[pdf.src != "distilled"]
filtered_source_pdf

,instruction,input,output,src,id
0,Human: Design melodic orchestrations by following the indicated chord progression.\n,"'A', 'D', 'E7', 'A', 'E/G#', 'A', 'Bm', 'A7/C#', 'D', 'E7', 'A', 'A', 'D', 'A', 'A', 'D', 'A', 'A', 'D', 'A', 'D', 'A/D#', 'E', 'A', 'D', 'A', 'A', 'D', 'A', 'E7' </s>","Assistant: X:1\nL:1/8\nM:2/2\nK:A\n CS ||""A"" A,2 CE FECE | AECE FECE | A,2 CE FECE |""D"" FAGF""E7"" EDCB, |""A"" A,2 CE FECE | AECE FEDC | \n""E/G#"" B,D""A""CE""Bm"" DF""A7/C#""E=G |""D"" FA""E7""GB""A"" cA A || f |""A"" eA""D""fA""A"" eAca | \n""A"" eA""D""fA""A"" e2 f/g/a |""A"" eA""D""fA""A"" eAce |""D"" df""A/D#""ca""E"" B2 d/e/f |""A"" eA""D""fA""A"" eAca | \n""A"" eA""D""fA""A"" e2 e/f/g | agag fedc |""E7"" BAGF EDC!D.S.!B, |] </s>",https://huggingface.co/datasets/sander-wood/irishman,0
18,Human: Assemble a piece of music by merging the alphabetic representation of the predetermined musical pattern and the provided melodic theme.\n,AA;X:1\nL:1/4\nM:2/2\nK:F\n['c A'] </s>,"Assistant: X:1\nL:1/4\nM:2/2\nK:F\n|:""F"" A3/2 B/ c A | G2 F2 |""Gm"" G3/2 A/ G D |""C7"" G3 z |""F"" A3/2 B/ c A |""Dm"" G2 F2 | \n""Gm"" G A""C7"" G3/2 F/ |""F"" F3 z ::""Bb"" B A B c |""Dm"" d2 A2 |""F"" c A G F |""C7"" G3 z | \n""F"" A3/2 B/ c A |""Dm"" G2 F2 |""Gm"" G A""C7"" G3/2 F/ |""F"" F3 z :| </s>",https://huggingface.co/datasets/sander-wood/irishman,18
19,Human: Construct music employing the alphabetic notation of the prescribed musical structure and the motif that has been supplied.\n,A;X:1\nL:1/8\nM:3/4\nK:C\n['c B'] </s>,Assistant: X:1\nL:1/8\nM:3/4\nK:C\n G2 | c2 G2 c2 | e4 dc | d2 G2 d2 | f4 ed | e2 cd ef | g2 fe dc | dB G2 F2 | (F E3) GG | c2 e2 dc | \n{e} d2 (cB) (AG) | c3 e (dc) |{c} B4 GG | (AB) c2 (BA) | (Bc) d2 e^f | g2 (dc) (BA) | G4 dd | \n d2 (cB) AG | d3 e f2 | e2 (dc) (BA) | e4 e2 | a2 (ed) (cB) | c B A B c A | d e f e d c | \n B A G A B G | c B c e f | g3 f e2 | f2 e2 d2 | c4 || </s>,https://huggingface.co/datasets/sander-wood/irishman,19
20,Human: Produce melodies following the designated musical arrangement and the given motif.\n,"'Binary', 'Sectional: Verse/Chorus'];X:1\nL:1/8\nM:4/4\nK:D\n['a2 fa g2 eg fd d/d/d dAFA', 'gfed caaf' </s>",Assistant: X:1\nL:1/8\nM:4/4\nK:D\n a2 fa g2 eg | fd d/d/d dAFA | BG G/G/G cA A/A/A | faaf g/f/e ce | a2 fa g2 eg | fd d/d/d dAFA | \n BG G/G/G cA A/A/A |1 fage defg :|2 fage d2 ed |: caaf gfed | caaf g/g/g fg | ad d/d/d agfg | \n abaf gfed | caaf gfed | caaf g/g/g fg | abaf gece |1 eddc d2 ed :|2 eddc defg || </s>,https://huggingface.co/datasets/sander-wood/irishman,20
21,Human: Examine the piece of music to locate the repeating motif from start to finish.\n,X:1\nL:1/8\nM:6/8\nK:G\n D | G3 G2 g | dBG GAB | cGE EFG | AFD DEF | G3 G2 g | dBG GAB | cEA TF3 | G3 G2 D | G3 G2 g | \n dBG GAB | c>de dcB | AFD DEF | G3 G2 g | dBG GAB | cEA TF3 | G3 G2 |: e | dBd dBd | g>ag Tf2 e | \n dBd dcB | AFD DFD | dBd dBd | g>ag Tf2 e | dcB AGF | G3 G2 :| </s>,Assistant: G3 G2 </s>,https://huggingface.co/datasets/sander-wood/irishman,21
...,...,...,...,...,...
1139455,"Human: Develop harmonies in line with the preset musical arrangement, integrating the supplied motif.\n","'Binary', 'Sectional: Verse/Chorus'];X:1\nL:1/8\nM:4/4\nK:C\n['Acc f2', 'F3 GCC GCC AF/G/A/F/ BGG A/B/cc d/e/ff' </s>","Assistant: X:1\nL:1/8\nM:4/4\nK:C\n[M:6/8][K:F] eae eec f Caa a""^the 1850"" .eeg e a!p! eg f e fa ae ee aa""^McDermott""[Q:1/8=80] .c | \n AFF Acc | f2 F fed | e/f/gc dcB | Acc f2 g | ab/a/g ab/a/g | fdd d2 e/f/ | gcg ga/g/f | \n ec/d/e f2 c | AFF cA/B/c/A/ | FA/B/c/A/ F2 f | ecc dB/c/d/B/ | c/B/A/B/c/A/ G>AG | FAf fFf | \n gcg afa | agf ece | fFF F3 || GCC GCC | AF/G/A/F/ BGG | A/B/cc d/e/ff | e/f/gg ece | \n fc'b a/b/c'/b/a/g/ | fcf fcf | bg/a/b/g/ fd/e/f/d/ | Bc/d/g ece | f2 F fed | c2 A BAG | \n A/B/cc fg/f/e | f3- f3 |] </s>",https://huggingface.co/datasets/sander-wood/irishman,1139455
1139459,Human: Carefully analyze this music work and recognize the particular motif within every segment.\n,"X:1\nL:1/8\nQ:1/4=70\nM:4/4\nK:G

In [ ]:
def process_text(text):
  if text.startswith("Human: "):
    text = text[7:]
  text = text.replace("</s>", "")
  text = text.strip()
  return text

filtered_source_pdf["instruction"] = filtered_source_pdf["instruction"].apply(process_text)


<ipython-input-42-e6ec5ec05ac6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_source_pdf["instruction"] = filtered_source_pdf["instruction"].apply(process_text)


In [ ]:
def process_text(text):
  if text.startswith("Assistant: "):
    text = text[11:]
  text = text.replace("</s>", "")
  text = text.strip()
  return text

filtered_source_pdf["output"] = filtered_source_pdf["output"].apply(process_text)


<ipython-input-43-371bea42c63f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_source_pdf["output"] = filtered_source_pdf["output"].apply(process_text)


In [ ]:
def process_text(text):
  text = text.replace("</s>", "")
  text = text.strip()
  return text

filtered_source_pdf["input"] = filtered_source_pdf["input"].apply(process_text)

<ipython-input-44-008109e47e37>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_source_pdf["input"] = filtered_source_pdf["input"].apply(process_text)


In [ ]:
filtered_source_pdf

,instruction,input,output,src,id
0,Design melodic orchestrations by following the indicated chord progression.,"'A', 'D', 'E7', 'A', 'E/G#', 'A', 'Bm', 'A7/C#', 'D', 'E7', 'A', 'A', 'D', 'A', 'A', 'D', 'A', 'A', 'D', 'A', 'D', 'A/D#', 'E', 'A', 'D', 'A', 'A', 'D', 'A', 'E7'","X:1\nL:1/8\nM:2/2\nK:A\n CS ||""A"" A,2 CE FECE | AECE FECE | A,2 CE FECE |""D"" FAGF""E7"" EDCB, |""A"" A,2 CE FECE | AECE FEDC | \n""E/G#"" B,D""A""CE""Bm"" DF""A7/C#""E=G |""D"" FA""E7""GB""A"" cA A || f |""A"" eA""D""fA""A"" eAca | \n""A"" eA""D""fA""A"" e2 f/g/a |""A"" eA""D""fA""A"" eAce |""D"" df""A/D#""ca""E"" B2 d/e/f |""A"" eA""D""fA""A"" eAca | \n""A"" eA""D""fA""A"" e2 e/f/g | agag fedc |""E7"" BAGF EDC!D.S.!B, |]",https://huggingface.co/datasets/sander-wood/irishman,0
18,Assemble a piece of music by merging the alphabetic representation of the predetermined musical pattern and the provided melodic theme.,AA;X:1\nL:1/4\nM:2/2\nK:F\n['c A'],"X:1\nL:1/4\nM:2/2\nK:F\n|:""F"" A3/2 B/ c A | G2 F2 |""Gm"" G3/2 A/ G D |""C7"" G3 z |""F"" A3/2 B/ c A |""Dm"" G2 F2 | \n""Gm"" G A""C7"" G3/2 F/ |""F"" F3 z ::""Bb"" B A B c |""Dm"" d2 A2 |""F"" c A G F |""C7"" G3 z | \n""F"" A3/2 B/ c A |""Dm"" G2 F2 |""Gm"" G A""C7"" G3/2 F/ |""F"" F3 z :|",https://huggingface.co/datasets/sander-wood/irishman,18
19,Construct music employing the alphabetic notation of the prescribed musical structure and the motif that has been supplied.,A;X:1\nL:1/8\nM:3/4\nK:C\n['c B'],X:1\nL:1/8\nM:3/4\nK:C\n G2 | c2 G2 c2 | e4 dc | d2 G2 d2 | f4 ed | e2 cd ef | g2 fe dc | dB G2 F2 | (F E3) GG | c2 e2 dc | \n{e} d2 (cB) (AG) | c3 e (dc) |{c} B4 GG | (AB) c2 (BA) | (Bc) d2 e^f | g2 (dc) (BA) | G4 dd | \n d2 (cB) AG | d3 e f2 | e2 (dc) (BA) | e4 e2 | a2 (ed) (cB) | c B A B c A | d e f e d c | \n B A G A B G | c B c e f | g3 f e2 | f2 e2 d2 | c4 ||,https://huggingface.co/datasets/sander-wood/irishman,19
20,Produce melodies following the designated musical arrangement and the given motif.,"'Binary', 'Sectional: Verse/Chorus'];X:1\nL:1/8\nM:4/4\nK:D\n['a2 fa g2 eg fd d/d/d dAFA', 'gfed caaf'",X:1\nL:1/8\nM:4/4\nK:D\n a2 fa g2 eg | fd d/d/d dAFA | BG G/G/G cA A/A/A | faaf g/f/e ce | a2 fa g2 eg | fd d/d/d dAFA | \n BG G/G/G cA A/A/A |1 fage defg :|2 fage d2 ed |: caaf gfed | caaf g/g/g fg | ad d/d/d agfg | \n abaf gfed | caaf gfed | caaf g/g/g fg | abaf gece |1 eddc d2 ed :|2 eddc defg ||,https://huggingface.co/datasets/sander-wood/irishman,20
21,Examine the piece of music to locate the repeating motif from start to finish.,X:1\nL:1/8\nM:6/8\nK:G\n D | G3 G2 g | dBG GAB | cGE EFG | AFD DEF | G3 G2 g | dBG GAB | cEA TF3 | G3 G2 D | G3 G2 g | \n dBG GAB | c>de dcB | AFD DEF | G3 G2 g | dBG GAB | cEA TF3 | G3 G2 |: e | dBd dBd | g>ag Tf2 e | \n dBd dcB | AFD DFD | dBd dBd | g>ag Tf2 e | dcB AGF | G3 G2 :|,G3 G2,https://huggingface.co/datasets/sander-wood/irishman,21
...,...,...,...,...,...
1139455,"Develop harmonies in line with the preset musical arrangement, integrating the supplied motif.","'Binary', 'Sectional: Verse/Chorus'];X:1\nL:1/8\nM:4/4\nK:C\n['Acc f2', 'F3 GCC GCC AF/G/A/F/ BGG A/B/cc d/e/ff'","X:1\nL:1/8\nM:4/4\nK:C\n[M:6/8][K:F] eae eec f Caa a""^the 1850"" .eeg e a!p! eg f e fa ae ee aa""^McDermott""[Q:1/8=80] .c | \n AFF Acc | f2 F fed | e/f/gc dcB | Acc f2 g | ab/a/g ab/a/g | fdd d2 e/f/ | gcg ga/g/f | \n ec/d/e f2 c | AFF cA/B/c/A/ | FA/B/c/A/ F2 f | ecc dB/c/d/B/ | c/B/A/B/c/A/ G>AG | FAf fFf | \n gcg afa | agf ece | fFF F3 || GCC GCC | AF/G/A/F/ BGG | A/B/cc d/e/ff | e/f/gg ece | \n fc'b a/b/c'/b/a/g/ | fcf fcf | bg/a/b/g/ fd/e/f/d/ | Bc/d/g ece | f2 F fed | c2 A BAG | \n A/B/cc fg/f/e | f3- f3 |]",https://huggingface.co/datasets/sander-wood/irishman,1139455
1139459,Carefully analyze this music work and recognize the particular motif within every segment.,"X:1\nL:1/8\nQ:1/4=70\nM:4/4\nK:G\n""_VB"" AG | (E>A) (A/B/d) e2 (dB) | e2 (A>B) (AG) E2 | E>A (A/B/d) e2 (ge) | (e>d) (B>A) A2 || \n""^dolce"" e2 | (ge)(ga) b2 (ag) | (e>d) (e/f/g) (G>A) (GE) | (ge)(ga) b2 (ag) | e2 (ab) 

In [ ]:
# prompt: trainset validation split by stratified sampling on src

from sklearn.model_selection import train_test_split

# validation file size should be lesser than 1 mb
trainset, validationset = train_test_split(filtered_source_pdf, test_size=0.003, stratify=filtered_source_pdf['src'], random_state=42)

print("Trainset size:", len(trainset))
print("Validationset size:", len(validationset))


Trainset size: 382788
Validationset size: 1152


In [ ]:
trainset.groupby("src").count()

,instruction,input,output,id
src,,,,
http://kern.ccarh.org,10409,10409,10409,10409
https://github.com/sander-wood/deepchoir,33399,33399,33399,33399
https://huggingface.co/datasets/sander-wood/irishman,338980,338980,338980,338980


In [ ]:
validationset.groupby("src").count()

,instruction,input,output,id
src,,,,
http://kern.ccarh.org,31,31,31,31
https://github.com/sander-wood/deepchoir,101,101,101,101
https://huggingface.co/datasets/sander-wood/irishman,1020,1020,1020,1020


In [ ]:
# prompt: Save in train and validation set in jsonl format like {"messages": [{"role": "user","content": "User interaction n°1 contained in document n°2"},{"role": "assistant","content": "Bot interaction n°1 contained in document n°2"},{"role": "user","content": "User interaction n°2 contained in document n°1"},{"role": "assistant","content": "Bot interaction n°2 contained in document n°1"}]}
import json

def convert_to_jsonl(dataset, filename):
  with open(filename, "w") as outfile:
    for _, example in dataset.iterrows():
      messages = []
      if example["instruction"] is not None and len(example["instruction"].strip()) > 0:
        messages.append({"role": "system", "content": example["instruction"]})
      if example["input"] is not None and len(example["input"].strip()) > 0:
        messages.append({"role": "user", "content": example["input"]})
      messages.append({"role": "assistant", "content": example["output"]})
      jsonl_example = {"messages": messages}
      jsonl_string = json.dumps(jsonl_example)
      outfile.write(jsonl_string + "\n")

convert_to_jsonl(trainset, "train.jsonl")
convert_to_jsonl(validationset, "validation.jsonl")


In [ ]:
!ls -ltrh

total 263M
drwxr-xr-x  1 root root 4.0K Jun 20 18:46 sample_data
drwxr-xr-x 10 root root 4.0K Jun 23 16:31 mistral-finetune
-rw-r--r--  1 root root 262M Jun 23 17:24 train.jsonl
-rw-r--r--  1 root root 799K Jun 23 17:24 validation.jsonl


In [ ]:
!head -n1 train.jsonl

{"messages": [{"role": "system", "content": "Generate a piece of music. The title is KREIS KREIS KESSEL. It belongs to the genres of \"Kinder - Lied\"\" Reigen\"and \" Tanz - Spiel\". It is from Europa, Mitteleuropa, Deutschland, Erfurt. Its meter type is simple duple."}, {"role": "user", "content": ""}, {"role": "assistant", "content": "X:1\nT:KREIS KREIS KESSEL\nT:KREIS KREIS KESSEL\nL:1/8\nM:2/4\nI:linebreak $\nK:G\nV:1 treble nm=\"Voice\" snm=\"V\"\nV:1\n G2 B2 | d2 B2 | GA Bc | d2 B2 | dd BG | ee d2 | cc BB | AA G2 || %8"}]}


Uploaded `train.jsonl` and `validation.jsonl` to drive manually

In [5]:
# prompt: read jsonl file from drive

from google.colab import drive
import json

drive.mount('/content/drive')

train_data_loaded = []
with open('/content/drive/MyDrive/mistral-ft/train.jsonl', 'r') as f:
  for l in f.readlines():
        a = json.loads(l)
        train_data_loaded.append(a)

# Process the lines as needed.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
from mistralai.client import MistralClient
from google.colab import userdata


api_key = userdata.get('MISTRAL_API_KEY')
client = MistralClient(api_key=api_key)

In [7]:
??client.files.create

In [11]:
with open('/content/drive/MyDrive/mistral-ft/train.jsonl', "rb") as f:
    training_data = client.files.create(file=("train.jsonl", f))

with open('/content/drive/MyDrive/mistral-ft/validation.jsonl', "rb") as f:
    validation_data = client.files.create(file=("validation.jsonl", f))


MistralAPIException: Status: 422. Message: {"detail": "Invalid file format.", "description": "Found 2 errors in this file. You can view supported formats here: https://docs.mistral.ai/capabilities/finetuning. ", "errors": [{"message": "line contains invalid json: unexpected content after document: line 1 column 463", "line_number": 19010}, {"message": "line contains invalid json: unexpected content after document: line 1 column 371", "line_number": 345484}]}

In [ ]:
from mistralai.models.jobs import WandbIntegrationIn, TrainingParameters

wandb_api_key = userdata.get('WANDB_API_KEY')

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ultrachat_chunk_train.id],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=300,
        learning_rate=0.0001,
    ),
    integrations=[
        WandbIntegrationIn(
            project="mistral-ft",
            run_name="test",
            api_key=wandb_api_key,
        ).dict()
    ],
    dry_run=True
)

# Dataset Validation

In [ ]:
!git clone https://github.com/mistralai/mistral-finetune.git

Cloning into 'mistral-finetune'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 401 (delta 125), reused 94 (delta 94), pack-reused 259
Receiving objects: 100% (401/401), 210.17 KiB | 6.37 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [ ]:
!pip install -r mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.9/704.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu

In [ ]:
!touch mistral-finetune/__init__.py

In [ ]:
!chmod 777 mistral-finetune/__init__.py

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.1/296.1 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


modify example/7B.yaml
```
# data
data:
  instruct_data: "../train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "../validation.jsonl"  # Optionally fill

# model
model_id_or_path: "open-mistral-7b"  # Change to downloaded path
lora:
  rank: 64

# optim
seq_len: 32768
batch_size: 1
max_steps: 1
optim:
  lr: 6.e-5
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "./"  # Fill

wandb:
  project: "mistral-ft" # your wandb project name
  run_name: "trial" # your wandb run name
  key: "<fill-this>" # your wandb api key
  offline: False
```

In validate_data.py

```
import sys

sys.path.append(".")
```

In [12]:
!cd mistral-finetune && python utils/validate_data.py --train_yaml example/7B.yaml &> out.txt

/bin/bash: line 1: cd: mistral-finetune: No such file or directory


In [ ]:
!ls mistral-finetune/utils

extend_model_vocab.py  merge_lora.py		reformat_data.py
__init__.py	       reformat_data_glaive.py	validate_data.py


# Debug jsonl format

In [ ]:
train_data_loaded = []
with open("train.jsonl", "r") as f:
    for l in f.readlines():
        a = json.loads(l)
        train_data_loaded.append(a)

In [ ]:
trainset[trainset.instruction.str.contains("The title is KREIS KREIS KESSEL. It belongs to the genres of")]

,instruction,input,output,src,id
396606,"Generate a piece of music. The title is KREIS KREIS KESSEL. It belongs to the genres of ""Kinder - Lied"""" Reigen""and "" Tanz - Spiel"". It is from Europa, Mitteleuropa, Deutschland, Erfurt. Its meter type is simple duple.",,"X:1\nT:KREIS KREIS KESSEL\nT:KREIS KREIS KESSEL\nL:1/8\nM:2/4\nI:linebreak $\nK:G\nV:1 treble nm=""Voice"" snm=""V""\nV:1\n G2 B2 | d2 B2 | GA Bc | d2 B2 | dd BG | ee d2 | cc BB | AA G2 || %8",http://kern.ccarh.org,396606
731082,"Generate a piece of music. The title is KREIS KREIS KESSEL. It belongs to the genres of ""Kinder - Lied"""" Reigen""and "" Tanz - Spiel"". It is from Europa, Mitteleuropa, Deutschland, Weimar. Its meter type is simple duple.",,"X:1\nT:KREIS KREIS KESSEL\nT:KREIS KREIS KESSEL\nL:1/8\nM:2/4\nI:linebreak $\nK:F\nV:1 treble nm=""Voice"" snm=""V""\nV:1\n c2 d2 | c2 A z | cc dd | c2 A z | BB GG | cc A2 | d>c BA | GG F2 || %8",http://kern.ccarh.org,731082


In [14]:
with open('/content/drive/MyDrive/mistral-ft/train.jsonl', 'r') as f:
    lines = f.readlines()
    print(lines[19009])
    print(lines[345483])

{"messages": [{"role": "system", "content": "Develop melodic sequences using the assigned musical pattern and provided motif."}, {"role": "user", "content": "Only One Section;X:1\nL:1/8\nM:6/8\nK:F\n['A2 A']"}, {"role": "assistant", "content": "X:1\nL:1/8\nM:6/8\nK:F\n E | F2 E F2 A | c3 A2 A | B3 G2 G | A3 F2 C | F2 E F2 A | c3 A2 A | B2 B c2 E | F4 | A | \n c2 c A2 A | c2 c A2 A | Bc B A2 G | F2 E F2 C | F2 E F2 A | c2 c A2 A | B2 B G2 G | A2 A F2 x |]"}]}

{"messages": [{"role": "system", "content": "Compose a musical piece that mirrors the style and essence of Bach's famous works."}, {"role": "user", "content": ""}, {"role": "assistant", "content": "X:1\nT:Music21 Fragment\nT:Music21 Fragment\nT:Music21\nC:Music21\n%%score 1 2 3 4\nL:1/4\nM:4/4\nK:C\nV:1 treble \n%%MIDI program 0\n%%MIDI control 7 100\n%%MIDI control 10 64\nV:2 treble \n%%MIDI program 0\n%%MIDI control 7 100\n%%MIDI control 10 64\nL:1/8\nV:3 bass \n%%MIDI program 0\n%%MIDI control 7 100\n%%MIDI control 10 64\nV:4 b

In [ ]:
with open("train.jsonl", "r") as f:
    with open("train_new.jsonl", "w") as w:
        i = 0
        for l in f.readlines():
            i+=1
            if i != 19010 and i!=52227 and i!=345484:
                w.write(l)